In [1]:
#%pip install ipython-sql configparser pandas psycopg2 numpy

In [2]:
import psycopg2
import configparser
import pandas as pd
import numpy as np
%load_ext sql

### Get authentication from config file

In [3]:
config = configparser.ConfigParser()
config.read('../private.cfg')
DB_NAME_DEFAULT = config.get('SQL', 'DB_NAME_DEFAULT')
DB_USER = config.get('SQL', 'DB_USER')
DB_PASSWORD = config.get('SQL', 'DB_PASSWORD') 

### Connect to Database

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname={} user={} password={}".format(DB_NAME_DEFAULT, DB_USER, DB_PASSWORD))
conn.set_session(autocommit=True)
cur = conn.cursor()

### Create Database

In [5]:
cur.execute('DROP DATABASE IF EXISTS coffeeshops')
cur.execute("CREATE DATABASE coffeeshops WITH ENCODING 'utf8' TEMPLATE template0")

### Close connection

In [6]:
conn.close()

### Connect to newly created Database

In [7]:
conn = psycopg2.connect("host=127.0.0.1 dbname=coffeeshops user={} password={}".format(DB_USER, DB_PASSWORD))
cur = conn.cursor()

In [8]:
cur.execute(
'''
DROP TABLE IF EXISTS employee
'''
)
conn.commit()

In [9]:
cur.execute(
'''
CREATE TABLE IF NOT EXISTS employees
(
employee_id INT PRIMARY KEY,
first_name VARCHAR(50),
last_name VARCHAR(50),
email VARCHAR(50),
hire_date DATE,
shop_name VARCHAR(50),
salary INT
)
'''
)
conn.commit()

### Read in CSV file and insert into database

In [13]:
df = pd.read_csv('../Data/demo/coffeeshop.csv')
df.head()

,employee_id,first_name,last_name,email,hire_date,shop_name,salary
0,54733,Vonni,Davsley,vdavsley0@joomla.org,21/03/2016,Urban Grind,9253
1,49009,Cary,Brauninger,cbrauninger1@independent.co.uk,26/08/2014,Common Grounds,45036
2,59278,Gerianna,Tolcharde,NaN,06/09/2019,Common Grounds,37007
3,92214,Helene,Bealing,NaN,12/11/2022,Urban Grind,13118
4,32890,Vivien,McCrackem,NaN,12/02/2019,Common Grounds,59034


In [11]:
# need to fix date style
for i, row in df.iterrows():
    cur.execute(
        '''
            INSERT INTO employees
            (employee_id, first_name, last_name, email, hire_date, shop_name, salary)
            VALUES(%s, %s, %s, %s, %s, %s, %s)
        ''',
        row.tolist()
    )
    conn.commit()

DatetimeFieldOverflow: date/time field value out of range: "21/03/2016"
LINE 4: ...4733, 'Vonni', 'Davsley', 'vdavsley0@joomla.org', '21/03/201...
                                                             ^
HINT:  Perhaps you need a different "datestyle" setting.
